In [1]:
mydatadir = "/data1/peterr/bilingual_data_for_ABClf"
outdir = "/data1/peterr/bilingual_data_for_ABClf_output/tsv"
import os
import pandas as pd
from ABClf import load_lexicon, get_variant, count_variants, counts_to_category
cols = "URL_en    URL_is    sent_en    sent_is    bleualign_score    paragraph_id_en    paragraph_id_is    deferred_hash_en    deferred_hash_is          bifixer_hash    bifixer_score    bicleaner_ai_score".split()
lex = load_lexicon()


import parse
def get_domain(url:str) -> str:
    pattern = "{protocol}://{domain}/{rest}"
    p = parse.compile(pattern)
    modified_pattern = "{protocol}://{domain}/"
    mp = parse.compile(modified_pattern)
    try:
        parse_result = p.parse(url)
        domain = parse_result["domain"]
    except TypeError:
        parse_result = mp.parse(url)
        domain = parse_result["domain"]
    return domain



import sys
import csv

csv.field_size_limit(131072<<4)


131072

In [5]:

for file in ['en-mk.deduped.txt']:
#for file in os.listdir(mydatadir):
    if not file.endswith(".txt"):
        continue
    lang = file[3:5]
    print(lang)
    
    endpath = os.path.join(
            outdir,
            file+".ABClf_output.txt",
        )
    print(file)
    df = pd.read_csv(
        os.path.join(
            mydatadir,
            file),
        sep="\t",
        engine="python",
        quoting=3,
        on_bad_lines="warn")
    
    df.columns = [col.replace("_is", f"_{lang}") for col in cols]


    s = df.groupby("URL_en")["sent_en"].apply(" ".join)
    ndf = pd.DataFrame(s)
    ndf["variant_unbalanced"] = ndf.sent_en.apply(lambda s: get_variant(s, lex=lex))
    ndf["per_token_unbalanced"] = ndf.sent_en.apply(lambda s: count_variants(s, lex)[1])
    ndf["domain"] = [get_domain(url) for url in ndf.index.values]
    ndf["URL_en"] = ndf.index

    df = pd.merge(
            df,
            ndf[['variant_unbalanced', 'per_token_unbalanced', 'domain',]],
            how="left",
            left_on="URL_en",
            right_index=True,
            sort=False,
            suffixes=("_x", "_y"),
            copy=True,
            indicator=False,
            validate=None,
            )
    df["en_domain_level_variant_from_docs"] = "UNK"
    df["en_domain_level_variant_from_counts"] = "UNK"
    counts = df.domain.value_counts()
    suitable_domains = counts[counts >= 3].index.values
    l = len(suitable_domains)

    for i, domain in enumerate(suitable_domains):
        # print(f"Working on domain {i} of {l}", end="\r")
        # #Calculate a subset of all docs that fit in this domain:
        # subset = df.loc[df.domain == domain, ["variant_unbalanced", "per_token_unbalanced"]]

        # subset_counts = subset.variant_unbalanced.value_counts()
        # most_frequent = subset_counts.index[0]
        # if most_frequent == "UNK":
        #     try:
        #         most_frequent = subset_counts.index[1]
        #     except:
        #         pass
    
        # df.loc[df.domain == domain, "en_domain_level_variant_from_docs"] = most_frequent
        d = dict()
        for line in subset.per_token_unbalanced.values:
            if not line:
                continue
            for word, payload in line.items():
                variant, count = payload.get("variant"), payload.get("count")
                d[variant] = d.get(variant, 0) + 1
        variant = counts_to_category(d)
        df.loc[df.domain == domain, "en_domain_level_variant_from_counts"] = variant

    # df.to_csv(
    #     endpath,
    #     index=False,
    #     sep="\t"
    # )


mk
en-mk.deduped.txt


Temporal perfomance:
* Original: 36 min
* Sped up: 32 min
* Only doc level: 20 min
* Only word level: 13 min

In [6]:
df.columns

Index(['URL_en', 'URL_mk', 'sent_en', 'sent_mk', 'bleualign_score',
       'paragraph_id_en', 'paragraph_id_mk', 'deferred_hash_en',
       'deferred_hash_mk', 'bifixer_hash', 'bifixer_score',
       'bicleaner_ai_score', 'variant_unbalanced', 'per_token_unbalanced',
       'domain', 'en_domain_level_variant_from_docs',
       'en_domain_level_variant_from_counts'],
      dtype='object')